# Visual stimulation monitor synchronisation

Bonsai controls visual display and saves a log everytime it asks for a frame to be rendered. However there is an unknown delay (~2 frames) between this render frame event and the actual display time. Furthermore some frames are skipped.

To figure out which frame is displayed when we put a photodiode in front of the monitor and display a pseudo-random sequence of alternating grey value. 

This notebook show how we use the frame logger and the photodiode signal to determine exact frame identity at each point of time. It has 3 main steps

**1. Detect frames on photodiode signal**

**2. Cross-correlate frame with sequence to find expected lag**

**3. Match cross correlation results to frame logger**


### Load example data

This next section just loads one example recording to be used for the rest of the notebook.

Define the session we want:

In [ ]:
PROJECT = "hey2_3d-vision_foodres_20220101"
MOUSE = "PZAH8.2i"
SESSION = "S20230209"
RECORDING = "R174123_SpheresPermTubeRewardPlayback"
PROTOCOL = "SpheresPermTubeReward"
MESSAGES = "harpmessage.bin"

Load it

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

import flexiznam as flz
from cottage_analysis.io_module import harp

flm_sess = flz.get_flexilims_session(PROJECT)
sess = flz.get_entity(name=f"{MOUSE}_{SESSION}", flexilims_session=flm_sess)
recs = flz.get_children(
    parent_id=sess.id, flexilims_session=flm_sess, children_datatype="recording"
)
print(f"Found {len(recs)} recordings for {MOUSE}_{SESSION}")
recs[recs.protocol == PROTOCOL].head()

In [ ]:
data_root = flz.PARAMETERS["data_root"]
recording = flz.get_entity(
    name=f"{MOUSE}_{SESSION}_{RECORDING}", flexilims_session=flm_sess
)

harp_message, harp_ds = harp.load_harpmessage(
    recording,
    flexilims_session=flm_sess,
    conflicts="skip",
    di_names=None,
)
harp_csvs = harp_ds.extra_attributes["csv_files"]

In [ ]:
frame_log = pd.read_csv(harp_ds.path_full / harp_csvs["FrameLog"])
expected_sequence = (
    pd.read_csv(
        harp_ds.path_full / harp_csvs["random_sequence_5values_alternate"], header=None
    )
    .loc[:, 0]
    .values
)
step_values = frame_log.PhotoQuadColor.unique()
ao_time = harp_message["analog_time"]
photodiode = harp_message["photodiode"]
ao_sampling = 1 / np.mean(np.diff(ao_time))

print("Data loaded.")
print(
    "Recording is %d s long."
    % (frame_log.HarpTime.values[-1] - frame_log.HarpTime.values[0])
)

In [ ]:
list(harp_messages.keys())

# Normal usage

This is using the main master function:

In [ ]:
from cottage_analysis.preprocessing import find_frames

processed = flz.get_data_root("processed", project=PROJECT)


frame_rate = 144
frame_detection_height = 0.1
correlation_threshold = 0.8
relative_corr_thres = 0.02
expected_lag = 2.0 / frame_rate
maxlag = 3.0 / frame_rate
minimum_lag = 1.0 / frame_rate
num_frame_to_corr = 6
time_column = "HarpTime"
sequence_column = "PhotoQuadColor"
save_folder = (
    processed / PROJECT / MOUSE / SESSION / RECORDING / "diagnostics" / "frame_sync"
)
print(f"Saving to {save_folder}")

frames_df, db_dict = find_frames.sync_by_correlation(
    frame_log,
    ao_time,
    photodiode,
    time_column=time_column,
    sequence_column=sequence_column,
    num_frame_to_corr=num_frame_to_corr,
    maxlag=maxlag,
    expected_lag=expected_lag,
    frame_rate=frame_rate,
    correlation_threshold=correlation_threshold,
    relative_corr_thres=relative_corr_thres,
    frame_detection_height=frame_detection_height,
    minimum_lag=minimum_lag,
    do_plot=True,
    verbose=True,
    debug=True,
    save_folder=save_folder,
)

# Detailled description

How does it work? The alignment is made in 3 steps:

- detect frames
- crosscorrelated with expected sequence
- align results

## Detect frames

The frame detection is simple: filter a bit to smooth local extrema, `diff` to find fast changes and detect peaks on that `diff` trace. This should detect all frame borders. In between these borders, look for the `diff` minimum to find the frame peak (be it a maximum or a minium)

Detection can be done independently using `detect_frame_onset`.


## Create frame df

In [ ]:
photodiode_time = ao_time
pd_sampling = 1 / np.mean(np.diff(photodiode_time))

# Normalise photodiode signal
normed_pd = np.array(photodiode, dtype=float)
normed_pd -= np.quantile(normed_pd, 0.01)
normed_pd /= np.quantile(normed_pd, 0.99)
photodiode_signal = normed_pd

height = frame_detection_height

# First step: Frame detection


pd_sampling = 1 / np.mean(np.diff(ao_time))
out = find_frames.detect_frame_onset(
    photodiode=photodiode,
    frame_rate=frame_rate,
    photodiode_sampling=pd_sampling,
    highcut=frame_rate * 3,
    debug=True,
    height=height,
)
frame_borders, peak_index, db_dict = out

In [ ]:
# cut frame detected before the recording started
t0 = frame_log[time_column].iloc[0]
to_cut = photodiode_time[frame_borders].searchsorted(t0)
print(f"Cutting {to_cut} frames detected before the recording started")

frame_borders = frame_borders[to_cut:]
peak_index = peak_index[to_cut:]

In [ ]:
frame_skip = np.diff(frame_borders) > pd_sampling / frame_rate * 1.5
frames_df = pd.DataFrame(
    dict(
        onset_sample=frame_borders[:-1],
        offset_sample=frame_borders[1:],
        peak_sample=peak_index,
        include_skip=frame_skip,
    )
)
frames_df["onset_time"] = photodiode_time[frames_df.onset_sample]
frames_df["offset_time"] = photodiode_time[frames_df.offset_sample]
frames_df["peak_time"] = photodiode_time[frames_df.peak_sample]

# check if frames are detected after the presentation is over
after_last = frames_df["onset_time"] >= frame_log[time_column].iloc[-1]
print(f"Framed detected after the presentation is over: {after_last.sum()}")

In [ ]:
frame_log

In [ ]:
late_frames.columns

In [ ]:
end_of_presentation = frame_log[time_column].iloc[-1]
last_frame = frames_df["onset_time"].iloc[-1]
print(f"Presentation ends at {end_of_presentation:.2f} s")
print(f"Last frame detected at {last_frame:.2f} s")

delay = frames_df["onset_time"].iloc[-1] - frame_log[time_column].iloc[-1]
print(f"{after_last.sum()} frames detected after the last render time.")

b, e = photodiode_time.searchsorted([end_of_presentation - 1, last_frame + 20])
fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 1, 1)
ax.set_title("Recording ends before last frame detected")
ax.plot(
    photodiode_time[b:e] - end_of_presentation,
    photodiode_signal[b:e],
    label="photodiode",
)
ax.axvline(0, color="k", label="end of presentation")
late_frames = frames_df[frames_df.onset_time > end_of_presentation]
ax.scatter(
    late_frames.peak_time - end_of_presentation,
    photodiode_signal[late_frames.peak_sample],
    label="late frame",
    color="purple",
)
ax.legend(loc="best")
ax.set_xlabel("Time relative to end of presentation (s)")
ax.set_ylabel("Photodiode signal (a.u.)")
save_folder = None
if save_folder is not None:
    fig.savefig(Path(save_folder) / f"presentation_end_issue.png")

You can get an example of detection using `plot_frame_detection_report`. This will give you `num_examples * 2` figures. Half of them are selected on random frames, half are centered around a frame drop.

In [ ]:
ignore_errors = True
ndetected = len(frames_df)
npresented = len(frame_log)
if npresented < ndetected:
    msg = (
        f"Detected more frames ({ndetected}) than presented ({npresented})"
        "\n Check create_frame_df parameters"
    )
elif npresented > ndetected * 2:
    msg = (
        f"Dropped more than half of the frames ({npresented - ndetected} dropped)"
        "\n Check create_frame_df parameters"
    )
else:
    msg = None
if msg is not None:
    if ignore_errors:
        warnings.warn(msg)
    else:
        raise ValueError(msg)

In [ ]:
plot_window = np.array([-7.5, 7.5]) / frame_rate * pd_sampling
figs = find_frames.plot_frame_detection_report(
    border_index=frame_borders,
    peak_index=peak_index,
    debug_dict=db_dict,
    num_examples=1,
    plot_window=plot_window,
    photodiode=photodiode,
    frame_rate=frame_rate,
    photodiode_sampling=pd_sampling,
    highcut=frame_rate * 3,
)

# Crosscorrelation

After having detected frames we will try to find where each of them falls in the photodiode sequence. To do that, we start by normalising the photodiode signal between 0 and 1

In [ ]:
frames_df, db_di = find_frames.run_cross_correlation(
    frames_df,
    frame_log,
    photodiode_time,
    normed_pd,
    time_column,
    sequence_column,
    num_frame_to_corr,
    maxlag,
    expected_lag,
    frame_rate,
    verbose=True,
    debug=True,
    pd_sampling=pd_sampling,
)

In [ ]:
frames_df = find_frames._match_fit_to_logger(
    frames_df,
    correlation_threshold=correlation_threshold,
    relative_corr_thres=relative_corr_thres,
    minimum_lag=minimum_lag,
    verbose=True,
)

# Then interpolate the missing frames
find_frames.interpolate_sync(frames_df, verbose=True)
# and remove the last double detected frames
frames_df = find_frames._remove_double_frames(frames_df, verbose=True)

In [ ]:
db_dict.update(db_di)
fig = plt.figure()
ax = fig.add_subplot(2, 1, 1)
find_frames.plot_crosscorr_matrix(
    ax, db_dict["cc_dict"], db_dict["lags_sample"], frames_df
)
ax = fig.add_subplot(2, 1, 2)
find_frames.plot_crosscorr_matrix(
    ax, db_dict["cc_dict"], db_dict["lags_sample"], frames_df
)
xl = ax.get_xlim()
mid = (xl[0] + xl[1]) / 2
ax.set_xlim(mid - 100, mid + 100)
fig.savefig(save_folder / "crosscorr_matrix.png")

In [ ]:
# plt.plot(frames_df.closest_frame_log_index.values)

ok = frames_df.closest_frame_log_index.values >= (
    frames_df.closest_frame_log_index.iloc[-1] - 1
)

# plt.plot(frames_df.closest_frame_log_index.values)
ok.argmax()

In [ ]:
frame = frames_df.index[919623 - 150]
fig = find_frames.plot_one_frame_check(
    frame,
    frames_df,
    frame_log,
    real_time=photodiode_time,
    normed_pd=normed_pd,
    ideal_time=db_dict["ideal_time"],
    ideal_pd=db_dict["ideal_photodiode_trace"],
    ideal_seqi=db_dict["ideal_seqi_trace"],
    num_frame_to_corr=None,
)
fig.suptitle(
    f"Frame {frame} matching frame log "
    f"{frames_df.loc[frame, 'closest_frame']}\n"
    f"Is interpolated: {not frames_df.loc[frame, 'interpolation_seeds']}"
)
fig.savefig(save_folder / f"frame_{frame}_check.png")

### Idealised photodiode

Then we generate an idealised version of what the photodiode signal should be (had their been no frame drops).

In [ ]:
seq_trace, ideal_pd = find_frames.ideal_photodiode(
    time_base=ao_time,
    switch_time=frame_log["HarpTime"].values,
    sequence=frame_log["PhotoQuadColor"].values,
)

fig = plt.figure()
w = np.array([10000, 10100])
t0 = ao_time[w[0]]
plt.plot(ao_time[slice(*w)] - t0, normed_pd[slice(*w)], label="Normed photodiode")
plt.plot(
    ao_time[slice(*w)] - t0,
    seq_trace[slice(*w)],
    label="Sequence",
    color="grey",
    alpha=0.5,
)
plt.plot(ao_time[slice(*w)] - t0, ideal_pd[slice(*w)], label="Filtered sequence")
l = plt.legend()

### Data chunking

Now we want to run the crosscorrelation around each frame.

We need to take a chunk of data that is big enough but short enough. Five or 6 frames seems to get good unique match with the sequence. Use `num_frame_to_corr` to set that.

Then we need to shift the photodiode by a given lag and cut the same chunk of data to correlate. There is no point in testing all the shifts, we now it will be about 2 frames. So we have `expected_lag ~= int(2/frame_rate*ao_sampling)` (in samples). 

To make things reasonably fast we also limit the search to a 3 frames of lag (+/- around expected_lag). With `maxlag ~= int(3/frame_rate*ao_sampling)`

In [ ]:
num_frame_to_corr = 6
maxlag_samples = int(np.round(3 / frame_rate * ao_sampling))  # make it into samples
expected_lag_samples = int(
    np.round(2 / frame_rate * ao_sampling)
)  # make it into samples

Finally we need to decide if we take the chunk of data before the frame, centered on the frame or after the frame. The best choice depends on if there was a frame drop recently or not. So let's just do the 3.

In [ ]:
window = [
    np.array([-1, 1]) * maxlag_samples
    + np.array(w * num_frame_to_corr / frame_rate * ao_sampling, dtype="int")
    for w in [np.array([-1, 0]), np.array([-0.5, 0.5]), np.array([0, 1])]
]
# for bef window, we add 1 frame to have the current frame included
window[0] += int(1 / frame_rate * ao_sampling)
# for center window, we shift by 0.5 frame to center
window[1] += int(0.5 / frame_rate * ao_sampling)

example_frame = 5234
frame_sample = frame_borders[example_frame]
lab = ["bef", "center", "aft"]
t0 = ao_time[frame_sample]
for iw, w in enumerate(window):
    part = slice(*w + frame_sample)
    plt.plot(
        ao_time[part][maxlag_samples : -maxlag_samples + 1] - t0,
        normed_pd[part][maxlag_samples : -maxlag_samples + 1] + iw * 0.5,
        label=lab[iw],
    )
plt.axvspan(
    ao_time[frame_sample] - t0,
    ao_time[frame_borders[example_frame + 1]] - t0,
    alpha=0.5,
)
_ = plt.legend()

In [ ]:
maxlag_samples

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(
    frames_df.lag.values * 1000,
    bins=np.arange(frames_df.lag.min() * 1000, frames_df.lag.max() * 1000),
)

In [ ]:
bad = np.diff(frames_df.closest_frame.values) < 1
frames_df.iloc[1:][bad]

Add that to the dataframe

## Match cross correlation results to frame logger

Ideally, if there is no frame drop, it does not matter if we look at the frames perceeding or following the frame we want to sync. That should be most of the case.

In [ ]:
db = db_dict["debug_info"]
normed_pd = np.array(photodiode, dtype=float)
normed_pd -= np.quantile(normed_pd, 0.01)
normed_pd /= np.quantile(normed_pd, 0.99)
db.keys()

In [ ]:
rng = np.random.default_rng(102)
w = frames_df[frames_df.sync_reason == "photodiode matching"].index
random_select = [w[i] for i in rng.integers(len(w), size=10)]
bad = np.diff(frames_df.closest_frame.values) < 1
badi = np.where(bad)[0]
random_select = frames_df.iloc[badi[20] + np.array([0, 1, 2], dtype=int)].index
labels = ["bef", "center", "aft"]
num_frame_to_corr = 5
maxlag = int(5.0 / frame_rate * ao_sampling)
expected_lag = int(2.0 / frame_rate * ao_sampling)
window = [
    np.array([-1, 1]) * maxlag
    + np.array(w * num_frame_to_corr / frame_rate * ao_sampling, dtype="int")
    for w in [np.array([-1, 0]), np.array([-0.5, 0.5]), np.array([0, 1])]
]
seq_trace = db["seq_trace"]

for frame in random_select:
    # frame = frames_df[~good].index[num]
    # frame = frames_df.index[num]
    fseries = frames_df.loc[frame]
    on_s = fseries.onset_sample
    off_s = fseries.offset_sample
    on_t = fseries.onset_time
    off_t = fseries.offset_time
    w = np.array([-50, 50])
    vfdf = frames_df[
        (frames_df.onset_sample > w[0] + on_s)
        & (frames_df.offset_sample < w[1] + off_s)
    ]
    qc = np.array([fseries[["quadcolor_%s" % w for w in labels]]])
    best = fseries.crosscorr_picked
    fig = plt.figure(figsize=(7, 7))
    plt.gca().get_yaxis().set_visible(False)

    col = dict(bef="r", center="g", aft="b")
    for i in range(3):
        label = "Photodiode" if i == 1 else None
        plt.plot(
            ao_time[slice(*w + on_s)] - on_t,
            normed_pd[slice(*w + on_s)] + i,
            label=label,
            color="purple",
        )
        label = "Frame #%d" % frame if i == 1 else None
        plt.axvspan(0, off_t - on_t, color="purple", alpha=0.2, label=label)
        plt.plot(fseries.peak_time - on_t, fseries.photodiode, "o", color="purple")

    vlog = frame_log[
        (frame_log.HarpTime > w[0] / ao_sampling + on_t - fseries.lag_bef)
        & (frame_log.HarpTime < w[1] / ao_sampling + off_t)
    ]
    plt.plot(
        vlog.HarpTime.values - on_t,
        vlog.PhotoQuadColor - 1.5,
        drawstyle="steps-post",
        label="Render frame",
    )

    i = 0
    for win, lab in zip(window, ["bef", "center", "aft"]):
        cut_win = win + maxlag * np.array([1, -1], dtype=int)
        l = fseries["lag_%s" % lab]
        part = seq_trace[slice(*win + on_s)]
        cut_part = seq_trace[slice(*cut_win + on_s)]
        x = normed_pd[slice(*win + on_s)][maxlag : -maxlag + 1]

        plt.plot(
            ao_time[slice(*win + on_s)] - on_t + l,
            part + i,
            alpha=0.75,
            lw=2,
            color=col[lab],
        )
        plt.plot(
            ao_time[slice(*win + on_s)][maxlag : -maxlag + 1] - on_t,
            x + i,
            alpha=0.5,
            lw=4,
            ls="--",
            color=col[lab],
        )

        cl = fseries["closest_frame_%s" % lab]
        plt.plot(
            frame_log.iloc[cl].HarpTime - on_t,
            frame_log.iloc[cl].PhotoQuadColor - 1.5 + i / 6,
            "o",
            color=col[lab],
        )
        if lab == best:
            plt.plot(
                frame_log.iloc[cl].HarpTime - on_t,
                frame_log.iloc[cl].PhotoQuadColor - 1.5 + i / 6,
                "o",
                mfc="None",
                mec="k",
                ms=10,
                mew=2,
            )
            plt.plot(
                frame_log.iloc[cl].HarpTime - on_t + l,
                frame_log.iloc[cl].PhotoQuadColor + i,
                "o",
                color="k",
            )
            plt.plot(
                ao_time[slice(*cut_win + on_s)] - on_t + l,
                cut_part + i,
                alpha=1,
                lw=1,
                color="k",
                label="Selected match",
            )

        i += 1
        plt.title("%s" % fseries.onset_sample)

    plt.legend(loc="lower right")

In [ ]:
# find what is the actual photodiode value and how does it depend on previous value

df = pd.DataFrame(frames_df.iloc[1:][["quadcolor", "photodiode"]]).reset_index()
bef = pd.DataFrame(frames_df.iloc[:-1][["quadcolor", "photodiode"]]).reset_index()
df["quadcolor_before"] = bef["quadcolor"]
df["photodiode_before"] = bef["photodiode"]
df.head()

In [ ]:
mat_data = (
    df.groupby(["quadcolor_before", "quadcolor"]).aggregate(np.nanmean).photodiode
)
n_data = df.groupby(["quadcolor_before", "quadcolor"]).aggregate(len).photodiode
m = mat_data.values.reshape((5, 5))
n = n_data.values.reshape((5, 5))
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.imshow(m.T, origin="lower")
cm = plt.colorbar()
plt.xlabel("quad n-1")
plt.ylabel("quad n")
plt.title("Photodiode")
plt.subplot(1, 3, 2)
plt.title("Difference")
plt.imshow((m - np.linspace(0, 1, 5)).T, origin="lower", cmap="RdBu_r")
plt.xlabel("quad n-1")
plt.ylabel("quad n")
cm = plt.colorbar()
plt.subplot(1, 3, 3)
plt.title("N transitions")
plt.imshow(n.T, origin="lower")
plt.xlabel("quad n-1")
plt.ylabel("quad n")
cm = plt.colorbar()

In [ ]:
fseries = frames_df.loc[4713]
fseries

In [ ]:
t0 = fseries.offset_time
frame_log["HarpTime"][1940:1955] - fseries.lag_aft - t0

In [ ]:
frames_df.sync_reason.value_counts()

# Divers stuff

Figures to explain things for my lab meeting (09/11/2022)

## Sequence principle

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
b = 1000
w = 100
shift = 1
0
seq = seq_trace[b : b + w]
bad_seq = np.array(seq_trace[b + shift : b + w + shift])
bad_seq[int(w / 3) : int(w / 3 + w / 3 * 0.6)] = bad_seq[int(w / 3)]
ax.plot((ao_time[b : b + w] - ao_time[b]) * 1e3, bad_seq)
ax.plot((ao_time[b : b + w] - ao_time[b]) * 1e3, seq + 1)
ax.set_xlabel("Time (ms)")
ax.yaxis.set_visible(False)
for w in ["top", "left", "right"]:
    ax.spines[w].set_visible(False)